In [17]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [25]:
sent_df = pd.read_csv("2_headword_headword_polarity.csv", header=None)[[0, 4]]
sentiment_dict = {row[0]: row[4] for _, row in sent_df.iterrows()}

def get_tfs(text):
    return dict(nltk.probability.FreqDist(([lemmatizer.lemmatize(w) for word in text if (w:=word.lower()).isalpha()])))

def get_sents(text):
    tf = get_tfs(nltk.word_tokenize(text))
    return (
        np.sum([sentiment_dict[word] * tf[word] for word in tf.keys() if word in list(sentiment_dict.keys())]) / 
        np.sum([tf[word] for word in tf.keys() if word in list(sentiment_dict.keys())]) 
    )

In [28]:
text = """
Hvilke hestesteder har du været "tilknyttet" , og ca hvornår

Her er en liste for mig

Lejre Rideklub ( ca 1985 - 1990)
Solrød Ridelub ( 1990 -1991 ) - red på en hest der stod opstaldet på den anden side af vejen
Følfods rideklub ( 1991 -1995 )
Gammeltoft rideklub ( 6 måneder sommer 1995)
Adamslyst ridecenter ( 1996 - 1997)
Baunehøj ridecenter ( red der ca 2 år hver anden uge i forbindelse med jeg gik På herlufsholm)
Vallensbæk ridecenter ( 2002-2004)
Dragør store magleby rideklub ( 2006 )

PT
Petersborg ( Høsterkøb) - Søster står der
Og hjemme hos mig selv J

Og så selvfølgelig Enghøjs ponyclub, hvor jeg er kommet siden 1985 og stadig kommer

Jeg har godt nok holdt til Mange steder inden for de sidste 22 år J
Man kan godt "læse" at jeg indtil nu udelukkende har været ½ parts rytter J

Hvor har i holdt til??
"""
get_sents(text)

0.3333333333333333